In [1]:
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
from torch import nn
from others.modules import *


In [2]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('data/train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('data/val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

# select a preset of images:

imgs_1 = noisy_imgs_1[:10000]/255
imgs_2 = noisy_imgs_2[:10000]/255
print(imgs_1.shape)

torch.Size([10000, 3, 32, 32])


In [3]:
class Model(object):
    def __init__(self):
        self.sequence = Sequential(Convolution(3, 20, kernel_size = 4, stride = 2),
                                   ReLU(),
                                   Convolution(20,20, kernel_size = 3, stride = 2),
                                   ReLU(),
                                   ConvolutionTransposed(20,20, kernel_size = 3, stride = 2),
                                   #Upsample(2),
                                   #Convolution(20,20, kernel_size = 3, stride = 1),
                                   ReLU(),
                                   ConvolutionTransposed(20,3, kernel_size = 4, stride = 2),
                                   #Upsample(2),
                                   #Upsample(3),
                                   #Convolution(20,3, kernel_size = 5, stride = 1),
                                   Sigmoid())
                                   

        self.criterion = MSE()
        self.optimizer = SGD(self.sequence, 1e-2)
        
        self.nb_samples = 10000
        self.epochs = 10
        print('Model Initialisation')

    def train(self,train_input,train_target):
    
        for epoch in range(self.epochs):

            if not epoch % 5: 
                    print('epoch',epoch)

            for n in range(self.nb_samples):

                if not n % 10000: 
                    print('sample: ', n)
                    print('weight',self.sequence.layers[0].weight[0,0,0])
                    print('grad',self.sequence.layers[0].grad[0,0,0])
                #print('img', train_input[n:n+1].shape)

                predict_img = self.sequence.forward(train_input[n:n+1])
                #print('optimizer zero grad')
                self.optimizer.zero_grad()
                #print('prediction', predict_img.shape)
                loss = self.criterion.forward(predict_img, train_target[n:n+1])
                #print('loss backward', self.criterion.backward())
                backward_img = self.sequence.backward(self.criterion.backward())
                #if not n % 100:
                    #print('loss', loss)
                    #print('grad conv0',self.sequence.layers[0].grad[0,0,0])
                    #print('grad sigmoid',self.sequence.layers[8].grad[0,0,0,0])
                    #print('grad upsample',self.sequence.layers[7].grad[0,0,0,0])
                    #print('grad relu',self.sequence.layers[6].grad[0,0,0,0])
                    #print('*********************************************************************************')
                self.optimizer.step()
        

        return backward_img
    
    def predict(self,test_input):
        prediction = torch.empty(test_input.shape)
            
        for n in range(test_input.shape[0]):
            prediction[n:n+1] = self.sequence.forward(test_input[n:n+1])

        return prediction
        

In [4]:
#img = torch.randn(1, 3, 32, 32)
#img_error = torch.randn(1, 3, 32, 32)

model = Model()

output = model.train(imgs_1, imgs_2)



print('out img', output.shape)



Model Initialisation
epoch 0
sample:  0
weight tensor([-0.0541, -0.2905,  0.2118, -0.2578])
grad tensor([2.7252e+20, 1.6635e+16, 1.0396e+21, 1.1329e+33])
sample:  0
weight tensor([-0.2936, -0.3715,  0.0837, -0.3454])
grad tensor([0., 0., 0., 0.])
sample:  0
weight tensor([-0.2935, -0.3715,  0.0837, -0.3453])
grad tensor([0., 0., 0., 0.])
sample:  0
weight tensor([-0.2936, -0.3715,  0.0836, -0.3454])
grad tensor([0., 0., 0., 0.])
sample:  0
weight tensor([-0.2936, -0.3715,  0.0836, -0.3454])
grad tensor([0., 0., 0., 0.])
epoch 5
sample:  0
weight tensor([-0.2936, -0.3715,  0.0836, -0.3454])
grad tensor([0., 0., 0., 0.])
sample:  0
weight tensor([-0.2936, -0.3715,  0.0836, -0.3454])
grad tensor([0., 0., 0., 0.])
sample:  0
weight tensor([-0.2937, -0.3716,  0.0835, -0.3455])
grad tensor([0., 0., 0., 0.])
sample:  0
weight tensor([-0.2937, -0.3716,  0.0835, -0.3455])
grad tensor([0., 0., 0., 0.])
sample:  0
weight tensor([-0.2937, -0.3716,  0.0835, -0.3455])
grad tensor([0., 0., 0., 0.])
o

In [5]:
def psnr ( denoised , ground_truth ) :
        # Peak Signal to Noise Ratio : denoised and ground_truth have range [0 , 1]
        mse = torch.mean (( denoised - ground_truth ) ** 2)
        return -10 * torch.log10 ( mse + 10** -8)

In [6]:
print(noisy_imgs.shape[0])

prediction = model.predict(noisy_imgs)


print(prediction.shape)

1000
torch.Size([1000, 3, 32, 32])


In [7]:
img_denoise_psnr = psnr(prediction ,clean_imgs )

In [8]:
print("psnr", img_denoise_psnr)

psnr tensor(5.1170)
